In [23]:
import pandas as pd
import numpy as np
import re
import nltk
import os
import joblib
import warnings
from typing import Dict, List, Tuple, Union, Any
import sys

# Machine Learning imports
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

# Visualization (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# NLTK
from nltk.corpus import stopwords

# Suppress warnings
warnings.filterwarnings('ignore')


# Download NLTK data
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')



In [25]:
import numpy as np 
import pandas as pd 
import os


file_path = '/kaggle/input/phishing-email-dataset/CEAS_08.csv'
initial_df = pd.read_csv(file_path)
initial_df.head()


,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1


In [26]:
df = initial[['body', 'label']]
df.head()


,body,label
0,"Buck up, your troubles caused by small dimensi...",1
1,\nUpgrade your sex and pleasures with these te...,1
2,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1
3,Would anyone object to removing .so from this ...,0
4,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1


In [27]:
print('shape.........')
print(df.shape)
print('info.....')
print(df.info())
print('isNull......')
print(df.isnull().sum())

shape.........
(39154, 2)
info.....
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39154 entries, 0 to 39153
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   body    39154 non-null  object
 1   label   39154 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 611.9+ KB
None
isNull......
body     0
label    0
dtype: int64


#  Feature Extraction

In [29]:
class EmailFeatureExtractor:

    keywords_dir = '/kaggle/input/keywordsfile/'
    if keywords_dir not in sys.path:
        sys.path.insert(0, keywords_dir)

    from keywords import (
        BUSINESS_KEYWORDS,
        REAL_ESTATE_KEYWORDS,
        PERSONAL_KEYWORDS,
        TECHNICAL_KEYWORDS,
        LEGAL_KEYWORDS,
        ENERGY_KEYWORDS,
        FINANCIAL_KEYWORDS,
        URGENCY_KEYWORDS,
        MALICIOUS_OFFER_KEYWORDS
    )

    def __init__(self, training_data=None):
        self.stop_words = set(stopwords.words('english'))
        self.exclusive_to_phishing = set()
        self.exclusive_to_ham = set()
        self.exclusive_phish_urls = set()
        self.exclusive_ham_urls = set()

        # If training data provided, learn exclusive patterns
        if training_data is not None:
            self.learn_exclusive_patterns(training_data)

    def learn_exclusive_patterns(self, df):
        print("Learning exclusive patterns from training data...")

        # 1: Extract words for Phishing (Label 1)
        df_phishing = df[df['label'] == 1]
        phishing_words = []
        for body in df_phishing['body'].dropna():
            phishing_words.extend(self.extract_clean_words(str(body)))
        unique_phishing = set(phishing_words)

        # 2: Extract words for Ham (Label 0)
        df_ham = df[df['label'] == 0]
        ham_words = []
        for body in df_ham['body'].dropna():
            ham_words.extend(self.extract_clean_words(str(body)))
        unique_ham = set(ham_words)

        # Words in Phishing that NEVER appear in Ham
        self.exclusive_to_phishing = unique_phishing - unique_ham

        # Words in Ham that NEVER appear in Phishing
        self.exclusive_to_ham = unique_ham - unique_phishing

        # URL extraction
        temp_df = df.copy()
        temp_df['extracted_url_list'] = temp_df['body'].apply(self.extract_urls)

        phishing_urls = []
        for url_list in temp_df[temp_df['label'] == 1]['extracted_url_list']:
            phishing_urls.extend(url_list)
        unique_phishing_urls = set(phishing_urls)

        ham_urls = []
        for url_list in temp_df[temp_df['label'] == 0]['extracted_url_list']:
            ham_urls.extend(url_list)
        unique_ham_urls = set(ham_urls)

        self.exclusive_phish_urls = unique_phishing_urls - unique_ham_urls
        self.exclusive_ham_urls = unique_ham_urls - unique_phishing_urls

        print(f"Learned {len(self.exclusive_to_phishing)} exclusive phishing words")
        print(f"Learned {len(self.exclusive_to_ham)} exclusive ham words")
        print(f"Learned {len(self.exclusive_phish_urls)} exclusive phishing URLs")
        print(f"Learned {len(self.exclusive_ham_urls)} exclusive ham URLs")

    def extract_clean_words(self, text):
        if not isinstance(text, str):
            return []

        # 1. Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # 2. Remove numbers and special characters
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # 3. Convert to lowercase
        words = text.lower().split()

        # 4. Filter out "helping words"
        meaningful_words = [w for w in words if w not in self.stop_words and len(w) > 2]

        return meaningful_words

    def extract_urls(self, text):
        """Extract URLs from text"""
        if not isinstance(text, str):
            return []

        url_pattern = r'https?://\S+|www\.\S+'
        urls = re.findall(url_pattern, text)
        return urls

    def detect_money_patterns(self, text):
        """Detect money patterns in text"""
        if not text or pd.isna(text):
            return 0, 0

        text_lower = str(text).lower()

        # Currency Symbols
        currency_symbols = (
            r"\$|£|€|¥|₹|؋|ƒ|₼|Br|BZ\$|\$b|KM|P|лв|R\$|៛|₡|kn|₱|Kč|kr|Q|¢|Ft|₨|₦|C\$|Gs|B\/\.|S\/\.|lei|₽|₪|Rp|﷼|₭|ден|RM|₮"
        )

        # Currency Names
        currency_names = (
            r"dollar|doller|usd|us dollars?|bucks?|"
            r"euro|eur|"
            r"pound|gbp|"
            r"rupee|rupees?|rs|inr|pkr|lkr|"
            r"yen|jpy|yuan|rmb|"
            r"riyal|rial|sar|qar|omr|"
            r"dirham|aed|"
            r"naira|ngn|"
            r"peso|mxn|php|"
            r"rube|rouble|rub|"
            r"shekel|ils|"
            r"ringgit|myr|"
            r"won|krw|"
            r"taka|bdt|"
            r"franc|chf|"
            r"lira|try|"
            r"rand|zar"
        )

        # Patterns
        symbol_pattern = rf"({currency_symbols})\s*[\d,]+\.?\d*"
        name_pattern = rf"([\d,]+\.?\d*\s*)?({currency_names})\s*[\d,]+\.?\d*"

        standard_count = len(re.findall(symbol_pattern, text_lower)) + len(re.findall(name_pattern, text_lower))

        # Obfuscated/text money
        obfuscated_patterns = [
            r'\bgrand\b', r'\bk\b', r'\bmil\b', r'\bbil\b',
            r'money|cash|fund|payment|transfer|account',
            r'win|prize|lottery',
            r'usd|gbp|eur|jpy|inr|cad',
        ]
        obfuscated_count = sum(len(re.findall(p, text_lower)) for p in obfuscated_patterns)

        return standard_count, obfuscated_count

    def extract_malicious_patterns(self, text: str) -> dict:

        if not isinstance(text, str):
            text = ""

        text_lower = text.lower()
        words = text_lower.split()
        total_chars = len(text)
        total_words = len(words) if len(words) > 0 else 1

        # 1. URL Features (Malicious emails often have many or suspicious URLs)
        urls = re.findall(r'https?://\S+|www\.\S+', text_lower)
        url_count = len(urls)

        # Check for numeric IPs (e.g., http://192.168.1.1)
        ip_in_url = 1 if re.search(r'https?://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', text_lower) else 0

        # 2. Urgency and Malicious Keywords
        urgency_score = sum(1 for word in words if word in self.URGENCY_KEYWORDS)
        offer_score = sum(1 for word in words if word in self.MALICIOUS_OFFER_KEYWORDS)
        business_score = sum(1 for word in words if word in self.BUSINESS_KEYWORDS)

        # 3. Punctuation & Style (Phishing often uses extreme punctuation)
        exclamation_count = text.count('!')
        dollar_count = text.count('$')
        caps_count = sum(1 for c in text if c.isupper())
        caps_ratio = caps_count / total_chars if total_chars > 0 else 0

        # 4. Email Header Deception
        is_fake_reply = 1 if text_lower.startswith(('re:', 'fwd:')) else 0

        # Compile Features
        malicious_features = {
            'text_length': total_chars,
            'word_count': total_words,
            'url_count': url_count,
            'url_density': url_count / total_words,
            'ip_in_url': ip_in_url,
            'urgency_score': urgency_score,
            'offer_score': offer_score,
            'business_score': business_score,
            'exclamation_density': exclamation_count / total_words,
            'dollar_density': dollar_count / total_words,
            'caps_ratio': caps_ratio,
            'is_fake_reply': is_fake_reply,
            'avg_word_length': total_chars / total_words
        }

        return malicious_features

    def extract_features_from_email(self, email_body):
 
        if not isinstance(email_body, str):
            email_body = str(email_body)

        features = {}

        # 1. Exclusive word features
        words = set(self.extract_clean_words(email_body))
        features['has_exclusive_phish_word'] = 1 if (words & self.exclusive_to_phishing) else 0
        features['has_exclusive_ham_word'] = 1 if (words & self.exclusive_to_ham) else 0

        # 2. Exclusive URL features
        urls = set(self.extract_urls(email_body))
        features['has_exclusive_phish_url'] = 1 if (urls & self.exclusive_phish_urls) else 0
        features['has_exclusive_ham_url'] = 1 if (urls & self.exclusive_ham_urls) else 0

        # 3. URL count
        features['url_count'] = len(urls)

        # 4. Body length
        features['body_len'] = len(email_body)

        # 5. Money detection
        standard, obfuscated = self.detect_money_patterns(email_body)
        features['money_standard_count'] = standard
        features['money_obfuscated_count'] = obfuscated

        # 6. Temporal features
        features['has_dates'] = 1 if re.search(
            r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}|'
            r'\d{1,2}/\d{1,2}/\d{2,4}|\d{4}-\d{2}-\d{2}',
            email_body, re.IGNORECASE
        ) else 0

        features['has_times'] = 1 if re.search(
            r'\b\d{1,2}:\d{2}\s*[ap]m?|\b\d{1,2}:\d{2}',
            email_body, re.IGNORECASE
        ) else 0

        features['timestamp_count'] = features['has_dates'] + features['has_times']

        # 7. Domain-specific features
        domain_patterns = {
            'is_business': '|'.join(self.BUSINESS_KEYWORDS),
            'is_personal': '|'.join(self.PERSONAL_KEYWORDS),
            'is_financial': '|'.join(self.FINANCIAL_KEYWORDS),
            'is_legal': '|'.join(self.LEGAL_KEYWORDS),
            'is_technical': '|'.join(self.TECHNICAL_KEYWORDS),
            'is_energy': '|'.join(self.ENERGY_KEYWORDS),
            'is_real_estate': '|'.join(self.REAL_ESTATE_KEYWORDS),
        }

        for feature_name, pattern in domain_patterns.items():
            features[feature_name] = 1 if re.search(pattern, email_body, re.IGNORECASE) else 0

        # 8. NEW: Malicious pattern features
        malicious_features = self.extract_malicious_patterns(email_body)
        features.update(malicious_features)

        return features

    def extract_features_from_dataframe(self, df):
        print("Extracting features from DataFrame...")

        # Make a copy to avoid modifying original
        df_features = df.copy()

        # 1. Exclusive word features
        def check_exclusive_words(text):
            if not isinstance(text, str):
                return 0, 0
            words = set(self.extract_clean_words(text))
            has_phish_word = 1 if (words & self.exclusive_to_phishing) else 0
            has_ham_word = 1 if (words & self.exclusive_to_ham) else 0
            return has_phish_word, has_ham_word

        # 2. Exclusive URL features
        def check_exclusive_urls(text):
            if not isinstance(text, str):
                return 0, 0
            urls = set(self.extract_urls(text))
            has_phish_url = 1 if (urls & self.exclusive_phish_urls) else 0
            has_ham_url = 1 if (urls & self.exclusive_ham_urls) else 0
            return has_phish_url, has_ham_url

        # Apply word features
        word_feats = df_features['body'].apply(check_exclusive_words)
        df_features['has_exclusive_phish_word'] = word_feats.apply(lambda x: x[0])
        df_features['has_exclusive_ham_word'] = word_feats.apply(lambda x: x[1])

        # Apply URL features
        url_feats = df_features['body'].apply(check_exclusive_urls)
        df_features['has_exclusive_phish_url'] = url_feats.apply(lambda x: x[0])
        df_features['has_exclusive_ham_url'] = url_feats.apply(lambda x: x[1])

        # 3. Additional features
        df_features['url_count'] = df_features['body'].apply(lambda x: len(self.extract_urls(str(x))))
        df_features['body_len'] = df_features['body'].apply(lambda x: len(str(x)))

        # 4. Money detection
        money_results = df_features['body'].apply(self.detect_money_patterns)
        df_features['money_standard_count'] = money_results.apply(lambda x: x[0])
        df_features['money_obfuscated_count'] = money_results.apply(lambda x: x[1])

        # 5. Temporal features
        df_features['has_dates'] = df_features['body'].str.contains(
            r'\b(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{1,2}|'
            r'\d{1,2}/\d{1,2}/\d{2,4}|\d{4}-\d{2}-\d{2}',
            case=False, na=False
        ).astype(int)

        df_features['has_times'] = df_features['body'].str.contains(
            r'\b\d{1,2}:\d{2}\s*[ap]m?|\b\d{1,2}:\d{2}',
            case=False, na=False
        ).astype(int)

        df_features['timestamp_count'] = df_features['has_dates'].astype(int) + df_features['has_times'].astype(int)

        # 6. Domain features
        domain_mappings = {
            'is_business': self.BUSINESS_KEYWORDS,
            'is_personal': self.PERSONAL_KEYWORDS,
            'is_financial': self.FINANCIAL_KEYWORDS,
            'is_legal': self.LEGAL_KEYWORDS,
            'is_technical': self.TECHNICAL_KEYWORDS,
            'is_energy': self.ENERGY_KEYWORDS,
            'is_real_estate': self.REAL_ESTATE_KEYWORDS,
        }

        for feature_name, keywords in domain_mappings.items():
            pattern = '|'.join(keywords)
            df_features[feature_name] = df_features['body'].str.contains(
                pattern, case=False, na=False, regex=True
            ).astype(int)

        # 7.Extract malicious patterns for each email
        malicious_features_list = df_features['body'].apply(self.extract_malicious_patterns)

        # Convert list of dicts to DataFrame and merge
        malicious_features_df = pd.DataFrame(malicious_features_list.tolist())
        df_features = pd.concat([df_features, malicious_features_df], axis=1)


        return df_features

# MODEL TRAINING 

In [31]:

# SECTION 1: MODEL TRAINER 


def train_model(model, X_train, y_train, X_test, y_test, model_name):
    """Train and evaluate a model"""
    print(f"\n--- Training and evaluating {model_name} ---")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print(f"\n{model_name} - Classification Report:")
    print(classification_report(y_test, y_pred))

    print(f"\n{model_name} - Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    roc_auc = roc_auc_score(y_test, y_proba)
    print(f"\n{model_name} - ROC AUC Score: {roc_auc:.4f}")

    return roc_auc, model


# SECTION 2: MAIN TRAINING FUNCTION 


def train_phishing_model(df):

    print("PHISHING DETECTION MODEL TRAINING")

    df = df[['body', 'label']].copy()

    print("Step 2: Training feature extractor...")
    extractor = EmailFeatureExtractor(training_data=df)

    print("Step 3: Extracting features...")
    df_features = extractor.extract_features_from_dataframe(df)



    # Display some extracted features
    print(" Sample of extracted features:")
    feature_cols = [c for c in df_features.columns if c not in ['body', 'label']]
    print(f"   Total features: {len(feature_cols)}")


    # Step 4: Prepare data for training
    print("Step 4: Preparing training data...")

    # Identify binary and continuous columns for scaling
    binary_columns = []
    continuous_count_columns = []

    # Drop the 'extracted_url_list' column if it exists
    if 'extracted_url_list' in df_features.columns:
        df_features = df_features.drop(columns=['extracted_url_list'])


    for col in df_features.columns:
        if col in ['body', 'label']:
            continue

        if pd.api.types.is_numeric_dtype(df_features[col]):
            unique_values = df_features[col].nunique()
            if unique_values == 2 and set(df_features[col].unique()) == {0, 1}:
                binary_columns.append(col)
            elif unique_values > 2 or (unique_values == 2 and not (set(df_features[col].unique()) == {0, 1})):
                continuous_count_columns.append(col)

    print(f"   Binary features: {len(binary_columns)}")
    print(f"   Continuous features: {len(continuous_count_columns)}")
    # display(df_features.head())
    # Scale continuous features
    df_scaled = df_features.copy()
    scaler = RobustScaler()

    if continuous_count_columns:
        df_scaled[continuous_count_columns] = scaler.fit_transform(df_scaled[continuous_count_columns])
        print(f"   Scaled {len(continuous_count_columns)} continuous features")

    # Step 5: Split data
    X = df_scaled.drop(columns=['body', 'label'])
    y = df_scaled['label']

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"Data split:")
    print(f"   Training set: {X_train.shape[0]} samples, {X_train.shape[1]} features")
    print(f"   Test set: {X_test.shape[0]} samples, {X_test.shape[1]} features")


        # Get model type
    model_types = ['logistic', 'random_forest', 'xgboost', 'svm', 'gradient_boosting']
    print(f"\nAvailable model types: {', '.join(model_types)}")
    model_type = input("Enter model type (default: logistic): ").strip().lower()

    if not model_type or model_type not in model_types:
        model_type = 'logistic'


    # Step 6: Train model
    print(f"Step 5: Training {model_type} model...")

    model_map = {
        'logistic': LogisticRegression(random_state=42, solver='liblinear', max_iter=1000),
        'random_forest': RandomForestClassifier(random_state=42, n_estimators=100),
        'xgboost': XGBClassifier(random_state=42, n_estimators=100, use_label_encoder=False, eval_metric='logloss'),
        'svm': SVC(random_state=42, probability=True),
        'gradient_boosting': GradientBoostingClassifier(random_state=42, n_estimators=100)
    }

    if model_type not in model_map:
        print(f" Unknown model type: {model_type}")
        print(f"   Available: {list(model_map.keys())}")
        model_type = 'logistic'

    model = model_map[model_type]
    model_name = model_type.replace('_', ' ').title()

    # Train and evaluate
    roc_auc, trained_model = train_model(model, X_train, y_train, X_test, y_test, model_name)

    # Step 7: Save all components
    print(f"Step 6: Saving model components...")

    # Create a components dictionary
    components = {
        'extractor': extractor,
        'model': trained_model,
        'scaler': scaler,
        'feature_columns': X_train.columns.tolist(),
        'binary_columns': binary_columns,
        'continuous_columns': continuous_count_columns,
        'model_type': model_type,
        'roc_auc': roc_auc
    }

    # Save to file
    model_filename = f'phishing_model_{model_type}.joblib'
    joblib.dump(components, model_filename)

    print(f" Model saved to: {model_filename}")
    print(f"   ROC AUC: {roc_auc:.4f}")
    print(f"   Features: {len(X_train.columns)}")

    # Step 8: Feature importance (if available)
    if hasattr(trained_model, 'feature_importances_'):
        print(f"\n Feature Importance (Top 10):")
        feature_importance = pd.DataFrame({
            'feature': X_train.columns,
            'importance': trained_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(feature_importance.head(10).to_string(index=False))

    print(" TRAINING COMPLETE!")

    return components


# SECTION 3: MAIN EXECUTION BLOCK


def main():
    train_phishing_model(df)

if __name__ == "__main__":

   main()




PHISHING DETECTION MODEL TRAINING
Step 2: Training feature extractor...
Learning exclusive patterns from training data...
Learned 24261 exclusive phishing words
Learned 112505 exclusive ham words
Learned 3742 exclusive phishing URLs
Learned 24773 exclusive ham URLs
Step 3: Extracting features...
Extracting features from DataFrame...
 Sample of extracted features:
   Total features: 31
Step 4: Preparing training data...
   Binary features: 15
   Continuous features: 14
   Scaled 14 continuous features
Data split:
   Training set: 31323 samples, 31 features
   Test set: 7831 samples, 31 features

Available model types: logistic, random_forest, xgboost, svm, gradient_boosting


Enter model type (default: logistic):  


Step 5: Training logistic model...

--- Training and evaluating Logistic ---

Logistic - Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3462
           1       1.00      1.00      1.00      4369

    accuracy                           1.00      7831
   macro avg       1.00      1.00      1.00      7831
weighted avg       1.00      1.00      1.00      7831


Logistic - Confusion Matrix:
[[3453    9]
 [   0 4369]]

Logistic - ROC AUC Score: 0.9997
Step 6: Saving model components...
 Model saved to: phishing_model_logistic.joblib
   ROC AUC: 0.9997
   Features: 31
 TRAINING COMPLETE!


# Mail Pridiction 


In [34]:
class PhishingPredictor:

    def __init__(self, model_path=None):

        if model_path:
            self.load_model(model_path)
        else:
            self.components = None
            self.is_loaded = False

    def load_model(self, model_path):

        print(f"Loading model from {model_path}...")

        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model file not found: {model_path}")

        self.components = joblib.load(model_path)
        self.is_loaded = True

        print(f" Model loaded successfully!")
        print(f"   Model type: {self.components['model_type']}")

    def predict_single(self, email_text, threshold=0.5, return_features=False):
        """
        Predict if a single email is phishing.

        """
        if not self.is_loaded:
            raise ValueError("Model not loaded. Call load_model() first.")

        # Step 1: Extract features
        extractor = self.components['extractor']
        features = extractor.extract_features_from_email(email_text)

        # Step 2: Create DataFrame with correct column order
        features_df = pd.DataFrame([features])

        # Ensure all expected columns exist (fill missing with 0)
        for col in self.components['feature_columns']:
            if col not in features_df.columns:
                features_df[col] = 0

        # Reorder columns to match training
        features_df = features_df[self.components['feature_columns']]

        # Step 3: Scale continuous features
        if self.components['continuous_columns']:
            features_df[self.components['continuous_columns']] = self.components['scaler'].transform(
                features_df[self.components['continuous_columns']]
            )

        # Step 4: Make prediction
        model = self.components['model']
        probability = model.predict_proba(features_df)[0]

        # Step 5: Prepare result
        phishing_prob = probability[1]
        is_phishing = phishing_prob >= threshold

        result = {
            'is_phishing': bool(is_phishing),
            'phishing_probability': float(phishing_prob),
            'ham_probability': float(probability[0]),
            'threshold_used': threshold,
            'confidence': self._get_confidence_level(phishing_prob)
        }


        # Add features if requested
        if return_features:
            result['features'] = features

        return result


    def _get_confidence_level(self, probability):
        """Get confidence level based on probability"""

        if probability < 0.3:
            return "Low"
        elif probability < 0.6:
            return "Medium"
        else:
            return "High"

    def analyze_email(self, email_text, verbose=True):

        result = self.predict_single(email_text, return_features=True)

        if verbose:
            print("\n" + "="*60)
            print("🔍 EMAIL ANALYSIS REPORT")
            print("\n" + "="*60)
            # Display prediction
            if result['is_phishing']:
                print(f"\n🔴 PHISHING DETECTED!")
                print(f"   Probability: {result['phishing_probability']:.1%}")
                print(f"   Confidence: {result['confidence']}")
            else:
                print(f"\n🟢 LEGITIMATE EMAIL")
                print(f"   Legitimacy Probability: {result['ham_probability']:.1%}")
                print(f"   Confidence: {result['confidence']}")



            # Display important features
            if 'features' in result:
                print(f"\n📊 Key Features:")
                features = result['features']
                
                # Show top 5 non-zero features
                non_zero_features = {k: v for k, v in features.items() if v != 0}
                sorted_features = sorted(non_zero_features.items(), key=lambda x: abs(x[1]), reverse=True)

                for feat, val in sorted_features[:25]:  # Show top 10
                    feat_display = feat.replace('_', ' ').title()
                    print(f"   • {feat_display}: {val}")

        return result
        
def main():
    
    model_path = '/kaggle/working/phishing_model_logistic.joblib'
    # Load predictor
    predictor = PhishingPredictor(model_path)

    # Prediction options
    print("PREDICTION OPTIONS:")
    print("1. Predict single email")

    pred_choice = input("\nEnter choice (1): ").strip()

    if pred_choice == '1':
        email = input("\nEnter email text: ").strip()
        result = predictor.analyze_email(email)

    else:
        print("Invalid choice.")

if __name__ == "__main__":

   main()

Loading model from /kaggle/working/phishing_model_logistic.joblib...
 Model loaded successfully!
   Model type: logistic
PREDICTION OPTIONS:
1. Predict single email



Enter choice (1):  1

Enter email text:  Buck up, your troubles caused by small dimension will soon be over! Become a lover no woman will be able to resist http://whitedone.com/come. Even as Nazi tanks were rolling down the streets, the dreamersphilosopher or a journalist. He was still not sure.I do the same.



🔍 EMAIL ANALYSIS REPORT


🔴 PHISHING DETECTED!
   Probability: 100.0%
   Confidence: High

📊 Key Features:
   • Body Len: 269
   • Text Length: 269
   • Word Count: 45
   • Avg Word Length: 5.977777777777778
   • Has Exclusive Phish Word: 1
   • Url Count: 1
   • Is Technical: 1
   • Caps Ratio: 0.022304832713754646
   • Url Density: 0.022222222222222223
   • Exclamation Density: 0.022222222222222223
